In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from hydra import initialize, compose
from omegaconf import OmegaConf

initialize(version_base="1.3", config_path="../configs", job_name="train")
cfg = compose(config_name="train") # overrides=["db=mysql", "db.user=me"]

In [2]:
from pathlib import Path
from omegaconf import open_dict
import rootutils

with open_dict(cfg):
    cfg.paths.root_dir = str(rootutils.find_root(indicator=".project-root"))
    output_dir = Path(cfg.paths.root_dir) / "output"
    output_dir.mkdir(parents=True, exist_ok=True)
    cfg.paths.output_dir = str(output_dir)
    cfg.paths.work_dir = "."

In [3]:
print(OmegaConf.to_yaml(cfg))

task_name: train
tags:
- dev
train: true
test: true
ckpt_path: null
seed: null
data:
  _target_: nanogpt.data.gpt_datamodule.NanoGPTDataModule
  dataset_name: WikiText2
  vocab_size: 500
  block_size: 10
  batch_size: 8
  min_len: 100
  force_tokenizer_retrain: false
  num_workers: 3
  pin_memory: false
model:
  _target_: nanogpt.models.gpt_module.NanoGPTLitModule
  optimizer:
    _target_: torch.optim.AdamW
    _partial_: true
    lr: 0.0001
  net:
    _target_: nanogpt.models.components.gpt.NanoGPT
    vocab_size: 500
    embedding_dim: 24
    block_size: 10
    n_heads: 4
    n_blocks: 1
  compile: false
callbacks:
  model_checkpoint:
    _target_: lightning.pytorch.callbacks.ModelCheckpoint
    dirpath: ${paths.output_dir}/checkpoints
    filename: epoch_{epoch:03d}
    monitor: val/acc
    verbose: false
    save_last: true
    save_top_k: 1
    mode: max
    auto_insert_metric_name: false
    save_weights_only: false
    every_n_train_steps: null
    train_time_interval: null
   

In [4]:
from nanogpt.train import extras, train, get_metric_value

extras(cfg)

/home/maaslak/PycharmProjects/lightning-hydra-template/.venv/lib/python3.10/site-packages/torchvision/io/image.py:13: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?


['dev']

[rank: 0] Field 'logger' not found in config. Skipping 'logger' config printing...


CONFIG
├── data
│   └── _target_: nanogpt.data.gpt_datamodule.NanoGPTDataModule                                                    
│       dataset_name: WikiText2                                                                                    
│       vocab_size: 500                                                                                            
│       block_size: 10                                                                                             
│       batch_size: 8                                                                                              
│       min_len: 100                                                                                               
│       force_tokenizer_retrain: false                                                                             
│       num_workers: 3                                                                                             
│       pin_memory: false                                                                                          
│                                                                                                                  
├── model
│   └── _target_: nanogpt.models.gpt_module.NanoGPTLitModule                                                       
│       optimizer:                                                                                                 
│         _target_: torch.optim.AdamW                                                                              
│         _partial_: true                                                                                          
│         lr: 0.0001                                                                                               
│       net:                                                                                                       
│         _target_: nanogpt.models.components.gpt.NanoGPT                                                          
│         vocab_size: 500                                                                                          
│         embedding_dim: 24                                                                                        
│         block_size: 10                                                                                           
│         n_heads: 4                                                                                               
│         n_blocks: 1                                                                                              
│       compile: false                                                                                             
│                                                                                                                  
├── callbacks
│   └── model_checkpoint:                                                                                          
│         _target_: lightning.pytorch.callbacks.ModelCheckpoint                                                    
│         dirpath: /home/maaslak/PycharmProjects/lightning-hydra-template/output/checkpoints                       
│         filename: epoch_{epoch:03d}                                                                              
│         monitor: val/acc                                                                                         
│         verbose: false                                                                                           
│         save_last: true                                                                                          
│         save_top_k: 1                                                                                            
│         mode: max                                                                                                
│         auto_insert_metric_name: false                                                                           
│         save_w

CONFIG
├── data
│   └── _target_: nanogpt.data.gpt_datamodule.NanoGPTDataModule                                                    
│       dataset_name: WikiText2                                                                                    
│       vocab_size: 500                                                                                            
│       block_size: 10                                                                                             
│       batch_size: 8                                                                                              
│       min_len: 100                                                                                               
│       force_tokenizer_retrain: false                                                                             
│       num_workers: 3                                                                                             
│       pin_memory: false                                                                                          
│                                                                                                                  
├── model
│   └── _target_: nanogpt.models.gpt_module.NanoGPTLitModule                                                       
│       optimizer:                                                                                                 
│         _target_: torch.optim.AdamW                                                                              
│         _partial_: true                                                                                          
│         lr: 0.0001                                                                                               
│       net:                                                                                                       
│         _target_: nanogpt.models.components.gpt.NanoGPT                                                          
│         vocab_size: 500                                                                                          
│         embedding_dim: 24                                                                                        
│         block_size: 10                                                                                           
│         n_heads: 4                                                                                               
│         n_blocks: 1                                                                                              
│       compile: false                                                                                             
│                                                                                                                  
├── callbacks
│   └── model_checkpoint:                                                                                          
│         _target_: lightning.pytorch.callbacks.ModelCheckpoint                                                    
│         dirpath: /home/maaslak/PycharmProjects/lightning-hydra-template/output/checkpoints                       
│         filename: epoch_{epoch:03d}                                                                              
│         monitor: val/acc                                                                                         
│         verbose: false                                                                                           
│         save_last: true                                                                                          
│         save_top_k: 1                                                                                            
│         mode: max                                                                                                
│         auto_insert_metric_name: false                                                                           
│         save_w

In [5]:
from nanogpt.models.gpt_module import NanoGPTLitModule

In [6]:
metric_dict, object_dict = train(cfg)

/home/maaslak/PycharmProjects/lightning-hydra-template/.venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
[rank: 0] No logger configs found! Skipping...


Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃    ┃ Name                                               ┃ Type               ┃ Params ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0  │ net                                                │ NanoGPT            │ 31.6 K │
│ 1  │ net.embedding                                      │ Embedding          │ 12.0 K │
│ 2  │ net.position_embedding                             │ Embedding          │    240 │
│ 3  │ net.attention_blocks                               │ Sequential         │  4.3 K │
│ 4  │ net.attention_blocks.0                             │ Block              │  4.3 K │
│ 5  │ net.attention_blocks.0.multiatt                    │ MultiHeadAttention │  1.7 K │
│ 6  │ net.attention_blocks.0.multiatt.heads              │ ModuleList         │  1.7 K │
│ 7  │ net.attention_blocks.0.multiatt.heads.0            │ SelfAttentionHead  │    432 │
│ 8  │ net.attention_blocks.0.multiatt.heads.0.value_proj │ Linear             │    144 │
│ 9  │ net.attention_blocks.0.multiatt.heads.0.key_proj   │ Linear             │    144 │
│ 10 │ net.attention_blocks.0.multiatt.heads.0.query_proj │ Linear             │    144 │
│ 11 │ net.attention_blocks.0.multiatt.heads.1            │ SelfAttentionHead  │    432 │
│ 12 │ net.attention_blocks.0.multiatt.heads.1.value_proj │ Linear             │    144 │
│ 13 │ net.attention_blocks.0.multiatt.heads.1.key_proj   │ Linear             │    144 │
│ 14 │ net.attention_blocks.0.multiatt.heads.1.query_proj │ Linear             │    144 │
│ 15 │ net.attention_blocks.0.multiatt.heads.2            │ SelfAttentionHead  │    432 │
│ 16 │ net.attention_blocks.0.multiatt.heads.2.value_proj │ Linear             │    144 │
│ 17 │ net.attention_blocks.0.multiatt.heads.2.key_proj   │ Linear             │    144 │
│ 18 │ net.attention_blocks.0.multiatt.heads.2.query_proj │ Linear             │    144 │
│ 19 │ net.attention_blocks.0.multiatt.heads.3            │ SelfAttentionHead  │    432 │
│ 20 │ net.attention_blocks.0.multiatt.heads.3.value_proj │ Linear             │    144 │
│ 21 │ net.attention_blocks.0.multiatt.heads.3.key_proj   │ Linear             │    144 │
│ 22 │ net.attention_blocks.0.multiatt.heads.3.query_proj │ Linear             │    144 │
│ 23 │ net.attention_blocks.0.ff                          │ FeadForward        │  2.5 K │
│ 24 │ net.attention_blocks.0.ff.layers                   │ Sequential         │  2.5 K │
│ 25 │ net.attention_blocks.0.ff.layers.0                 │ Linear             │  1.2 K │
│ 26 │ net.attention_blocks.0.ff.layers.1                 │ LayerNorm          │     96 │
│ 27 │ net.attention_blocks.0.ff.layers.2                 │ ReLU               │      0 │
│ 28 │ net.attention_blocks.0.ff.layers.3                 │ Linear             │  1.2 K │
│ 29 │ net.attention_blocks.0.ff.layers.4                 │ LayerNorm          │     48 │
│ 30 │ net.attention_blocks.0.ff.layers.5                 │ ReLU               │      0 │
│ 31 │ net.attention_blocks.0.ln                          │ LayerNorm          │     48 │
│ 32 │ net.ff                                             │ FeadForward        │  2.5 K │
│ 33 │ net.ff.layers                                      │ Sequential         │  2.5 K │
│ 34 │ net.ff.layers.0                                    │ Linear             │  1.2 K │
│ 35 │ net.ff.layers.1                                    │ LayerNorm          │     96 │
│ 36 │ net.ff.layers.2                                    │ ReLU               │      0 │
│ 37 │ net.ff.layers.3                                    │ Linear             │  1.2 K │
│ 38 │ net.ff.layers.4                                    │ LayerNorm          │     48 │
│ 39 │ net.ff.layers.5                                    │ ReLU               │      0 │
│ 40 │ net.proj                                           │ Linear             │ 12.5 K │
│ 41 │ train_loss                       

Trainable params: 31.6 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 31.6 K                                                                                               
Total estimated model params size (MB): 0

Output()

[rank: 0] 
Traceback (most recent call last):
  File "/home/maaslak/PycharmProjects/lightning-hydra-template/nanogpt/utils/utils.py", line 68, in wrap
    metric_dict, object_dict = task_func(cfg=cfg)
  File "/home/maaslak/PycharmProjects/lightning-hydra-template/nanogpt/train.py", line 87, in train
    trainer.fit(model=model, datamodule=datamodule, ckpt_path=cfg.get("ckpt_path"))
  File "/home/maaslak/PycharmProjects/lightning-hydra-template/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/trainer.py", line 544, in fit
    call._call_and_handle_interrupt(
  File "/home/maaslak/PycharmProjects/lightning-hydra-template/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py", line 44, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
  File "/home/maaslak/PycharmProjects/lightning-hydra-template/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/trainer.py", line 580, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/h

RuntimeError: Early stopping conditioned on metric `val/acc` which is not available. Pass in or modify your `EarlyStopping` callback to use any of the following: `train/loss`, `train/loss_step`, `val/loss`, `train/loss_epoch`

In [12]:
import torch
object_dict["model"].forward(torch.tensor([[1]]))

tensor([[  1,   0,  74, 475,  75, 444,  56,  12, 416,  51, 219, 204,  70,  83,
         472,  93, 332, 495,  64,  82, 297, 457, 291,  69, 285, 355, 455, 187,
         303, 349,  45,  12,  64,  81, 113, 213, 400, 450, 370, 409, 371, 140,
         350, 493,   4, 297, 188,   2, 404, 353,  43]])